## 1. Load HVA cicuit

In [1]:
from db_doi.utils_XXZ_compilation import *
from qibo import Circuit
L = 10
H_XXZ = XXZ_compilation_line(L,gateset="RZZ",delta=0.5)
E0,E1 = H_XXZ.eigenergies_ED()
warm_start_qasm_file = "example_results/circuit_qasm/Quantinuum_Example_HVA_warm_start.qasm"
with open(warm_start_qasm_file, 'r') as file:
    warm_start_qasm_str = file.read()
warm_start_circ = Circuit.from_qasm(warm_start_qasm_str)
# unify initial kwargs
c = Circuit(L)
warm_start_circ.init_kwargs = c.init_kwargs
warm_start_energy = H_XXZ.expectation(
                H_XXZ.backend.execute_circuit(circuit=warm_start_circ).state())
print("E0:", E0)
print("E1:", E1)
print("Warm start energy:", warm_start_energy)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
[Qibo 0.2.21|INFO|2025-08-19 14:07:00]: Using qibojit (numba) backend on /CPU:0


E0: -14.361002811946294
E1: -13.437060121401611
Warm start energy: -13.678712484079853


In [2]:
from db_doi.group_commutator_iteration import GroupCommutatorIteration, DoubleBracketRotationApproximationType
gci = GroupCommutatorIteration(deepcopy(H_XXZ), warm_start_circ, DoubleBracketRotationApproximationType.group_commutator_reduced )
print(gci.loss())
print(gci.double_bracket_rotation_approximation_type)

-13.678712484079853
DoubleBracketRotationApproximationType.group_commutator_reduced


## 2. Create DOI ansatz

In [ ]:
class DOI_magnetic_field(hamiltonians.SymbolicHamiltonian):
    def __init__(self, L, coef=None):
        self.L = L
        self.coef = coef
        if coef is None:
            self.coef = [1]*L
        super().__init__(sum([self.coef[i]*Z(i) for i in range(L)]))        
        self.gateset = "RZ"        
    
    def circuit(self,t, coef=None):
            import qibo
            c = qibo.Circuit(L)
            if coef is None:
                coef = self.coef
            for i in range(L):
                c.add(qibo.gates.RZ(i, coef[i]*t*2))
            return c
doi = DOI_magnetic_field(L)

In [ ]:
from scipy.optimize import minimize, minimize_scalar
import numpy as np

def cost_function(coef, step):
    doi.coef = coef
    return gci.loss(step, doi)

def optimize_step_coef(cost_function, c0, K, maxiter_s=10, maxiter_c=20, verbose=True):
    c_current = c0.copy()
    s_current = 0
    f_min = cost_function(c0, 0)
    for k in range(K):
        cost_fn_s = lambda s: cost_function(coef=c_current, step=s)
        res_s = minimize_scalar(cost_fn_s, bounds=(1e-4, 0.3), method="bounded",
                    options={"maxiter": maxiter_s, "disp":False} )
        if verbose:
            print(f"Iteration {k+1}: Minimum {res_s.fun} at step={res_s.x} in {res_s.nfev} evaluations")
        f_current = res_s.fun
        if f_current < f_min:
            f_min = res_s.fun
            s_current = res_s.x
        else:
            break
        cost_fn_c = lambda c: cost_function(coef=c, step=s_current)
        res_c = minimize(
                    cost_fn_c,
                    c0,
                    method="POWELL",
                    options={"disp": True, "maxiter": maxiter_c, "disp": False},
                    tol=1e-3,
                    )
        if verbose:
            print(f"             Minimum {res_c.fun} at coef={res_c.x} in {res_c.nfev} evaluations")
        f_current = res_c.fun
        if f_current < f_min:
            f_min = res_c.fun
            c_current = res_c.x
        else:
            break
    return c_current, s_current, f_min

In [109]:
c0 = [np.sin(x/L)/2 for x in range(L)]
# c0 = [0.5 for x in range(L)]
c, s, f = optimize_step_coef(cost_function, c0, 10)

Iteration 1: Minimum -13.738494149328385 at step=0.21495661476983174 in 8 evaluations
             Minimum -13.863711011865565 at coef=[0.55274202 0.50554253 0.59225415 0.31387697 0.3340342  0.32362951
 0.54527715 0.35712533 0.56670649 0.49653656] in 243 evaluations
Iteration 2: Minimum -13.871471385591684 at step=0.18400136961863722 in 8 evaluations
             Minimum -13.87609163578458 at coef=[0.60485247 0.54196227 0.67134705 0.33792433 0.40053749 0.3639736
 0.63589485 0.3800159  0.62648382 0.52425072] in 245 evaluations
Iteration 3: Minimum -13.878364104120319 at step=0.16849919287279536 in 8 evaluations
             Minimum -13.87988422988889 at coef=[0.64291784 0.55812524 0.71460923 0.35626626 0.43484144 0.3911675
 0.69158718 0.39424134 0.66603285 0.53993522] in 243 evaluations
Iteration 4: Minimum -13.880721795097246 at step=0.159522127185216 in 9 evaluations
             Minimum -13.881336693527077 at coef=[0.6476277  0.57586579 0.75749184 0.36346116 0.46705892 0.40323829
 0.

In [110]:
doi.coef = c
gci(s, doi)
print(gci.loss())

-13.88255543651492


In [111]:
from qibo import models
db_doi_1_qasm = models.Circuit.to_qasm(gci.preparation_circuit)
folder_path = "example_results/circuit_qasm/"
with open(folder_path+f"Quantinuum_Example_HVA_DB-DOI_k=1_optimized.qasm", "w") as file:
    file.write(db_doi_1_qasm)